In [ ]:
pip install scikit-surprise


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=2811609 sha256=14237d236de8ed816cdb049d262b6bd28dde0e08abcf45864107c1bcc1fa07b0
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [ ]:
import pandas as pd
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split

# Load the data
movies_df = pd.read_csv('Netflix_Dataset_Movie.csv')
ratings_df = pd.read_csv('Netflix_Dataset_Rating.csv')

# Create a utility matrix
utility_matrix = ratings_df.pivot_table(index='User_ID', columns='Movie_ID', values='Rating')

# Fill missing values with 0 (assuming no rating means a rating of 0)
utility_matrix = utility_matrix.fillna(0)

# Display the utility matrix
print(utility_matrix)
# Define a Reader to specify the rating scale (from 1 to 5)
reader = Reader(rating_scale=(1, 5))

# Create a Surprise Dataset object
data = Dataset.load_from_df(ratings_df[['User_ID', 'Movie_ID', 'Rating']], reader)

# Split the data into a training set and a test set (75% training, 25% testing)
trainset, testset = train_test_split(data, test_size=0.25, random_state=42)

# Build and train an SVD-based collaborative filtering model
model = SVD()
model.fit(trainset)

# Specify the User_ID for whom you want to make recommendations
user_id = 7  # Replace with the User_ID you want to recommend movies for

# Get a list of movies that the user has not yet rated
user_unrated_movies = ratings_df[~ratings_df['Movie_ID'].isin(trainset.all_items())]

# Predict ratings for the unrated movies
user_predictions = [(movie_id, model.predict(user_id, movie_id).est) for movie_id in user_unrated_movies['Movie_ID']]

# Sort the predicted ratings in descending order
user_predictions.sort(key=lambda x: x[1], reverse=True)

# Number of recommendations to provide
num_recommendations = 10  # Change this to the desired number of recommendations

# Keep track of recommended movie IDs
recommended_movie_ids = set()

# Print the recommended movie IDs and their predicted ratings
print("Recommended Movies with Predicted Ratings for User {}: ".format(user_id))
for movie_id, rating in user_predictions:
    if len(recommended_movie_ids) >= num_recommendations:
        break
    if movie_id not in recommended_movie_ids:
        recommended_movie_ids.add(movie_id)
        recommended_movie = movies_df[movies_df['Movie_ID'] == movie_id]['Name'].values[0]
        print("Movie ID: {}, Movie Name: {}, Predicted Rating: {:.2f}".format(movie_id, recommended_movie, rating))



Movie_ID  3    8    16   17   18   26   28   30   32   33   ...  273  275  \
User_ID                                                     ...             
6         0.0  4.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
7         0.0  0.0  0.0  0.0  0.0  5.0  0.0  4.0  0.0  0.0  ...  2.0  0.0   
79        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
97        0.0  0.0  0.0  0.0  0.0  0.0  0.0  4.0  0.0  0.0  ...  0.0  0.0   
134       0.0  0.0  0.0  0.0  0.0  0.0  4.0  4.0  0.0  5.0  ...  0.0  0.0   
...       ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
256054    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  4.0  0.0   
256057    0.0  0.0  0.0  4.0  0.0  0.0  3.0  2.0  0.0  0.0  ...  0.0  0.0   
256065    0.0  0.0  0.0  0.0  0.0  0.0  3.0  0.0  0.0  0.0  ...  0.0  0.0   
256067    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  ...  0.0  0.0   
256072    0.0  5.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   